In [1]:
import filepath_p2

In [2]:
# import filepath
import os

# 设置 DISPLAY
os.environ["DISPLAY"] = ":0"
from direct.showbase.ShowBase import ShowBase
from panda3d.core import WindowProperties

In [3]:
from panda3d.core import (
    NodePath,
    WindowProperties,
    Vec3,
    TextNode,
    PNMImage, Texture,
    CardMaker,Point2,
    NodePath, Camera, PerspectiveLens,
    Point3, LVector3f
)

In [4]:
from art.basic import (
    create_cube_node, create_sphere_node,
uv_curve_surface,create_colored_cube_node
)

In [5]:
from direct.showbase.ShowBase import ShowBase
   Vec4
from panda3d.core import (
    AmbientLight,
    DirectionalLight,
    PointLight,
    Shader,
    Vec3,
    Vec4
)
# Shader
# Application Class
# =================
class ShaderDemo(ShowBase):
    def __init__(self):
        # Call the base constructor
        ShowBase.__init__(self)
        self.sphere = create_sphere_node("sphere", lat_res=24, lon_res=24)
        self.sphere_np = NodePath(self.sphere)
        self.sphere_np.set_pos(0,5,0)
        self.sphere_np.reparent_to(self.render)

        self.ambient_light = self.render.attach_new_node(AmbientLight("AmbientLight"))
        self.ambient_light.node().set_color(Vec4(.2, .2, .2, 1))
        self.render.set_light(self.ambient_light)
        
        self.sun = self.render.attach_new_node(DirectionalLight("Sun"))
        self.sun.set_hpr(45, -45, 0)
        self.render.set_light(self.sun)
        
        self.green_light = self.render.attach_new_node(PointLight("GreenLight"))
        self.green_light.node().set_color(Vec4(0, 1, 0, 1))
        self.green_light.node().set_attenuation(Vec3(1, .1, .5))
        self.green_light.set_pos(0, 3, 0)
        self.render.set_light(self.green_light)
        


# Entry Point
# ===========
if __name__ == "__main__":
    ShaderDemo().run()


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4381.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SystemExit: 

/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
